In [13]:
import polars as pl
from db.model import Groups, Software, GroupSoftware
from miter.create_db import create_tactic_as_records, create_technique_as_records, create_mitigation_as_records, save_to_mysql

In [14]:
tactic_path = "/home/work/data/enterprise-attack-v15.1-tactics.xlsx"
datasource_path = "/home/work/data/enterprise-attack-v15.1-datasources.xlsx"
technique_path = "/home/work/data/enterprise-attack-v15.1-techniques.xlsx"
mitigation_path = "/home/work/data/enterprise-attack-v15.1-mitigations.xlsx"
relation_path = "/home/work/data/enterprise-attack-v15.1-relationships.xlsx"
group_path = "/home/work/data/enterprise-attack-v15.1-groups.xlsx"
software_path = "/home/work/data/enterprise-attack-v15.1-software.xlsx"

In [15]:
target_model = GroupSoftware

In [16]:
target_model.drop_table()
target_model.create_table()

In [17]:
COLUMN_MAPPING_SOFTWARE = {
    "ID": "software_id",
    "name": "name",
    "description": "description_en",
    # "description_jp": "description_jp",
    "url": "url",
    "created": "created",
    "last modified": "last_modified",
    "version": "version",
    "type": "type"
}

In [18]:
software_df = pl.read_excel(software_path, sheet_name="associated groups")

In [19]:
map_replace_map = {"source ID": "group_id", "target ID": 'software_id'}
software_df.head()

source ID,source name,source ref,source type,mapping type,target ID,target name,target ref,target type,mapping description,STIX ID,created,last modified
str,str,str,str,str,str,str,str,str,str,str,str,str
"""G1028""","""APT-C-23""","""intrusion-set--8332952e-b86b-4…","""group""","""uses""","""S0339""","""Micropsia""","""malware--8c050cea-86e1-4b63-bf…","""software""",null,"""relationship--edfebe9b-4387-47…","""26 March 2024""","""26 March 2024"""
"""G0099""","""APT-C-36""","""intrusion-set--c4d50cdf-87ce-4…","""group""","""uses""","""S0434""","""Imminent Monitor""","""tool--8f8cd191-902c-4e83-bf20-…","""software""","""(Citation: QiAnXin APT-C-36 Fe…","""relationship--3f010259-666c-40…","""05 May 2020""","""14 October 2020"""
"""G0006""","""APT1""","""intrusion-set--6a2e693f-24e5-4…","""group""","""uses""","""S0017""","""BISCUIT""","""malware--b8eb28e4-48a6-40ae-95…","""software""","""(Citation: Mandiant APT1)""","""relationship--19fce62c-ba70-4c…","""31 May 2017""","""20 August 2019"""
"""G0006""","""APT1""","""intrusion-set--6a2e693f-24e5-4…","""group""","""uses""","""S0025""","""CALENDAR""","""malware--5a84dc36-df0d-4053-9b…","""software""","""(Citation: Mandiant APT1)""","""relationship--7909f5a6-3924-42…","""31 May 2017""","""20 August 2019"""
"""G0006""","""APT1""","""intrusion-set--6a2e693f-24e5-4…","""group""","""uses""","""S0119""","""Cachedump""","""tool--c9cd7ec9-40b7-49db-80be-…","""software""","""(Citation: Mandiant APT1)""","""relationship--1d5e0da2-7741-4a…","""31 May 2017""","""20 August 2019"""


In [10]:

group_df = group_df.rename(COLUMN_MAPPING_SOFTWARE).select(list(COLUMN_MAPPING_SOFTWARE.values()))
group_df = group_df.with_columns(pl.Series("description_jp", [None]* len(group_df)))


In [12]:
save_to_mysql(target_model, group_df.to_dicts())

In [8]:
def save_to_mysql(table, records) -> None:
    table.insert_many(records).execute()